In [173]:
import pandas as pd
import os
import sys
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

from keras import backend as K
import tensorflow as tf
config = tf.ConfigProto(device_count = {'CPU': 1}, intra_op_parallelism_threads=7, inter_op_parallelism_threads=1)
session = tf.Session(config=config)
K.set_session(session)
import keras

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


import PconvNetPolimi as pcnp
import numpy as np
import gene4cancer as gc
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)
matplotlib.rcParams['font.size'] = 22
matplotlib.rcParams['xtick.labelsize'] = 22
matplotlib.rcParams['ytick.labelsize'] = 22
matplotlib.rcParams['axes.labelsize'] = 30
matplotlib.rcParams['axes.titlesize'] = 30

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
path_dm = '/home/nanni/Data/HiC/50kb/distance_matrix.npy'
d_m = np.load(path_dm)
idx_to_gene_path = '/home/nanni/Data/HiC/50kb/idx_to_gene.csv'
df_idx_to_gene = pd.read_csv(idx_to_gene_path,index_col=0,header = None)
gene_to_idx_path = '/home/nanni/Data/HiC/50kb/gene_to_idx.csv'
df_gene_to_idx = pd.read_csv(gene_to_idx_path,index_col=0,header = None)

In [23]:
path_data = '/home/nanni/Data/TCGA/tcga.tsv'
df_data = pd.read_csv(path_data,sep='\t')
filtered_df = df_data[df_data['sample_type'].isin(['Primary Solid Tumor','Solid Tissue Normal'])]
gene_dm = df_gene_to_idx.index.tolist()
X_normal, Y_normal, X_tumor, Y_tumor, gene_to_idx, idx_to_gene = gc.data_preprocessing.preprocess_data(filtered_df[filtered_df.columns[:7].tolist() + gene_dm])
X, Y, state = gc.data_preprocessing.assemble_normal_tumor(X_normal, Y_normal, X_tumor, Y_tumor)

/home/sakalouski/.conda/envs/artur/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Taking 300 random genes  

In [24]:
indexes = list(np.random.randint(low=0, high=X.shape[1], size = (300)))
X_300 = X[:,indexes]

### Splitting the data 

In [268]:
X_train, Y_train, X_test, Y_test = gc.data_preprocessing.split(X_300, Y)
#X_train, Y_train, X_cv, Y_cv = gc.data_preprocessing.split(X_train, Y_train)

### Getting the baseline 

In [269]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
classy = SVC()
classy.fit(X_train,Y_train)
y_pred = classy.predict(X_test)
rep = classification_report(Y_test, y_pred)
print(rep)
#print(confusion_matrix(Y_test, y_pred))

             precision    recall  f1-score   support

       BRCA       1.00      0.99      1.00       353
   COADREAD       1.00      0.98      0.99       124
       KIRC       1.00      0.96      0.98       186
       LIHC       1.00      0.98      0.99       129
       LUAD       0.07      0.05      0.06       164
       LUNG       0.26      0.37      0.31       329
       LUSC       0.00      0.00      0.00       172
       PRAD       1.00      0.99      1.00       175
       THCA       1.00      0.99      1.00       179

avg / total       0.69      0.70      0.69      1811



### Preparing data for DL 

In [371]:
kern_size = 5
d_m_cropped = d_m[indexes,:]
d_m_cropped = d_m_cropped[:,indexes]

order = []
for i in range(X_train.shape[1]):
    sortet = np.argsort(d_m_cropped[i,:])
    order.append(i)
    order.extend(sortet[1:kern_size])

print(order)
from random import shuffle
shuffle(order) #for random dm
print(order)
X_dl = X_300[:,order]

[0, 61, 178, 224, 196, 1, 17, 187, 60, 15, 2, 235, 68, 288, 154, 3, 192, 52, 143, 19, 4, 102, 119, 194, 62, 5, 14, 97, 82, 139, 6, 138, 105, 149, 242, 7, 106, 267, 72, 188, 8, 81, 101, 268, 297, 9, 100, 7, 260, 44, 10, 4, 78, 97, 139, 11, 174, 246, 232, 210, 12, 0, 224, 178, 73, 13, 85, 144, 64, 83, 14, 5, 82, 97, 64, 15, 146, 105, 165, 151, 16, 102, 4, 62, 226, 17, 255, 169, 201, 41, 18, 172, 202, 230, 197, 19, 193, 202, 99, 80, 20, 207, 98, 141, 244, 21, 117, 87, 274, 67, 22, 45, 256, 95, 136, 23, 181, 110, 226, 7, 24, 83, 188, 170, 131, 25, 97, 188, 78, 89, 26, 262, 72, 191, 267, 27, 6, 105, 146, 31, 28, 207, 98, 296, 297, 29, 282, 7, 49, 195, 30, 100, 49, 91, 136, 31, 216, 16, 181, 269, 32, 33, 158, 60, 150, 33, 32, 239, 89, 150, 34, 157, 223, 295, 19, 35, 53, 171, 225, 163, 36, 271, 94, 211, 280, 37, 168, 293, 51, 270, 38, 288, 203, 154, 68, 39, 45, 199, 5, 166, 40, 24, 83, 188, 170, 41, 17, 19, 23, 146, 42, 254, 250, 244, 198, 43, 159, 36, 280, 126, 44, 251, 116, 86, 221, 45, 95,

In [359]:
X_dl.shape

(6035, 1500)

In [360]:
from sklearn.preprocessing import StandardScaler
def str_to_int(Y):
    list_of_tissues = []
    for i in range(Y.shape[0]):
        e = Y[i]
        if e in list_of_tissues:
            Y[i] = list_of_tissues.index(e)
        else:
            list_of_tissues.append(e)
            Y[i] = list_of_tissues.index(e)
    Y = Y.astype(int)
    return Y

In [372]:
Y = str_to_int(Y)
X_train_dl, Y_train_dl, X_test_dl, Y_test_dl = gc.data_preprocessing.split(X_dl, Y)

Y_input_test = keras.utils.to_categorical(Y_test_dl, num_classes=len(np.unique(Y_test_dl)))
Y_input_train = keras.utils.to_categorical(Y_train_dl, num_classes=len(np.unique(Y_train_dl)))

scaler = StandardScaler()
X_train_dl = scaler.fit_transform(X_train_dl)
X_test_dl = scaler.transform(X_test_dl)

X_input_test = X_test_dl.reshape(X_test_dl.shape[0],X_test_dl.shape[1],-1)
X_input_train = X_train_dl.reshape(X_train_dl.shape[0],X_train_dl.shape[1],-1)

In [373]:
X_input_train, Y_input_train = pcnp.upsampling(X_input_train, Y_input_train)

In [374]:
np.unique(Y_train_dl, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([307, 858, 798, 381, 419, 292, 410, 385, 374]))

In [375]:
from keras.models import Model
from keras.layers import (Lambda, Flatten, Dropout, Dense, Input, BatchNormalization)
from keras.backend import floatx
def create_conv_model(X_train, Y_train, nb_filters, nb_neighbors, opt=None):
    
    nb_features = X_train.shape[1]
    data = Input(shape=(nb_features, 1), name="data", dtype=floatx())
    conv_layer = keras.layers.Conv1D(nb_neighbors, nb_filters, activation='relu', strides = nb_neighbors)(data)
    #conv_layer = keras.layers.Conv1D(nb_neighbors, nb_filters, activation='relu', strides = nb_neighbors)(conv_layer)
    flatt = Flatten()(conv_layer)
    drop = Dropout(0.25)(flatt)
    drop = Dense(units=16, activation='relu')(drop)
    drop = BatchNormalization()(drop)
    #drop = Dropout(0.25)(drop)
    output = Dense(units=Y_train.shape[1], activation="softmax", name='output')(drop)

    model = Model(inputs=data, outputs=output)
    from keras import optimizers
    opt = optimizers.Nadam(lr=1e-3)
    #opt = optimizers.SGD(lr = 1e-4)
    model.compile(optimizer=opt, loss='categorical_crossentropy')

    return model

In [376]:
batch_size = 100
model_name = 'Test'
model = create_conv_model(X_input_train, Y_input_train, 3, kern_size)

In [377]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-7, verbose=1)
checkpointer = ModelCheckpoint(filepath=model_name+'.h5', verbose=1, save_best_only=True)
EarlStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=12, verbose=0, mode='auto')

history = model.fit(X_input_train, Y_input_train, 
                      epochs = 300, 
                      batch_size = batch_size, 
                      verbose=1, 
                      validation_split = 0.3,
                      #validation_data=(X_input_cv, Y_input_cv),
                      callbacks=[reduce_lr, checkpointer, EarlStop],
                      shuffle=True)

Train on 5406 samples, validate on 2317 samples
Epoch 1/300
5406/5406 [==============================] - 2s - loss: 1.0106 - val_loss: 0.6243
Epoch 2/300
5406/5406 [==============================] - 0s - loss: 0.4770 - val_loss: 0.4378
Epoch 3/300
5406/5406 [==============================] - 0s - loss: 0.3835 - val_loss: 0.3748
Epoch 4/300
5406/5406 [==============================] - 0s - loss: 0.3350 - val_loss: 0.3607
Epoch 5/300
5406/5406 [==============================] - 0s - loss: 0.3116 - val_loss: 0.3161
Epoch 6/300
5406/5406 [==============================] - 0s - loss: 0.2922 - val_loss: 0.3080
Epoch 7/300
5406/5406 [==============================] - 0s - loss: 0.2758 - val_loss: 0.3062
Epoch 8/300
5406/5406 [==============================] - 0s - loss: 0.2673 - val_loss: 0.3102
Epoch 9/300
5406/5406 [==============================] - 0s - loss: 0.2576 - val_loss: 0.3126
Epoch 10/300
5406/5406 [==============================] - 0s - loss: 0.2525 - val_loss: 0.3136
Epoch 11/30

In [379]:
Y_pred = np.empty_like(Y_input_test)
for i in range(0,Y_pred.shape[0]):
    Y_pred[i,:] = model.predict(x = X_input_test[[i],:,:])

Y_pred_1d = pcnp.to_1d_labels(Y_pred)
rep = classification_report(Y_test_dl.astype(int),Y_pred_1d)
print(rep)

             precision    recall  f1-score   support

          0       1.00      0.99      1.00       124
          1       1.00      0.98      0.99       353
          2       0.21      0.12      0.15       329
          3       0.31      0.37      0.34       172
          4       0.98      0.99      0.99       186
          5       1.00      0.98      0.99       129
          6       0.40      0.70      0.51       164
          7       0.99      1.00      1.00       179
          8       0.99      1.00      0.99       175

avg / total       0.73      0.75      0.73      1811



In [370]:
print(confusion_matrix(Y_test.astype(int), Y_pred_1d))

[[123   0   0   1   0   0   0   0   0]
 [  0   0   0   0 184   1   1   0   0]
 [  0   0   0   1   0 128   0   0   0]
 [  0   1  31 144   0   0 153   0   0]
 [  0   0   0   0   0   0   0   0 175]
 [  0 349   0   2   2   0   0   0   0]
 [  0   0  77   4   0   0  83   0   0]
 [  0   0   0   0   0   0   0 178   1]
 [  0   0  77  92   1   1   1   0   0]]
